# Feedforward network to classify traffic signs using Keras

- Set the first layer to a Flatten() layer with input_shape set to (32, 32, 3).
- Set the second layer to a Dense() layer with an output width of 128.
- Use a ReLU activation function after the second layer.
- Set output layer width to 5, because for this data set there are only 5 classes.
- Use a softmax activation function after the output layer.

In [21]:
import pickle
import numpy as np
import tensorflow as tf

with open('small_train_traffic.p', mode='rb') as f:
    data = pickle.load(f)
with open('small_test_traffic.p', mode='rb') as testf:
    test_data = pickle.load(testf)

X_train, y_train = data['features'], data['labels']
X_normalized = np.array(X_train / 255.0 - 0.5)

X_test,  y_test  = test_data['features'], test_data['labels']
X_test2 = np.array(X_test / 255.0 - 0.5)

from sklearn.preprocessing import LabelBinarizer
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Flatten

model = Sequential()
model.add(Flatten(input_shape=(32, 32, 3)))
model.add(Dense(128))
model.add(Activation('relu'))
# model.add(Dense(5))
# model.add(Activation('softmax'))
model.add(Dense(5, activation='softmax'))

label_binarizer = LabelBinarizer()
y_one_hot = label_binarizer.fit_transform(y_train)
y_test_onehot = label_binarizer.fit_transform(y_test)

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics  =['accuracy'])
history = model.fit(X_normalized, 
                    y_one_hot, epochs=3,
                    validation_split=0.2)

print('----------------------------------------------------------------------')
score = model.evaluate(X_test2, y_test_onehot, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 80 samples, validate on 20 samples
Epoch 1/3
80/80 [==============================] - 0s 5ms/step - loss: 1.2441 - acc: 0.3625 - val_loss: 0.8747 - val_acc: 0.6500
Epoch 2/3
80/80 [==============================] - 0s 221us/step - loss: 0.9460 - acc: 0.6500 - val_loss: 0.5806 - val_acc: 0.8500
Epoch 3/3
80/80 [==============================] - 0s 194us/step - loss: 0.6182 - acc: 0.7000 - val_loss: 0.5649 - val_acc: 0.7500
----------------------------------------------------------------------
Test loss: 0.8161932826042175
Test accuracy: 0.6499999761581421


# Convolutions
- Build from the previous network.
- Add a convolutional layer with 32 filters, a 3x3 kernel, 
    - and valid padding before the flatten layer.
- Add a ReLU activation after the convolutional layer.
- Train for 3 epochs again, should be able to get over 50% accuracy.

In [33]:
from keras.layers.convolutional import Conv2D
from keras.losses import categorical_crossentropy
from keras.optimizers import Adadelta

model2 = Sequential()
model2.add(Conv2D(32, (3,3), input_shape=(32, 32, 3), activation='relu'))
model2.add(Flatten())
model2.add(Dense(128, activation='relu'))
model2.add(Dense(5, activation='softmax'))

label_binarizer = LabelBinarizer()
y_one_hot = label_binarizer.fit_transform(y_train)
y_test_onehot = label_binarizer.fit_transform(y_test)

model2.compile(loss=categorical_crossentropy,
              optimizer=Adadelta(),
              metrics  =['accuracy'])
history = model2.fit(X_normalized, 
                    y_one_hot, epochs=3,
                    validation_split=0.2)

print('----------------------------------------------------------------------')
score = model2.evaluate(X_test2, y_test_onehot, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 80 samples, validate on 20 samples
Epoch 1/3
80/80 [==============================] - 1s 9ms/step - loss: 1.2890 - acc: 0.3750 - val_loss: 0.7974 - val_acc: 0.7500
Epoch 2/3
80/80 [==============================] - 0s 1ms/step - loss: 0.8118 - acc: 0.6250 - val_loss: 0.9243 - val_acc: 0.7000
Epoch 3/3
80/80 [==============================] - 0s 1ms/step - loss: 0.9469 - acc: 0.5750 - val_loss: 0.5656 - val_acc: 0.8500
----------------------------------------------------------------------
Test loss: 0.8366739153862
Test accuracy: 0.699999988079071


# Pooling

- Build from the previous network
- Add a 2x2 max pooling layer immediately following your convolutional layer.

In [37]:
from keras.layers.pooling import MaxPooling2D

model3 = Sequential()
model3.add(Conv2D(32, (3,3), input_shape=(32, 32, 3), activation='relu'))
model3.add(MaxPooling2D((2, 2)))
model3.add(Flatten())
model3.add(Dense(128, activation='relu'))
model3.add(Dense(5, activation='softmax'))

label_binarizer = LabelBinarizer()
y_one_hot = label_binarizer.fit_transform(y_train)
y_test_onehot = label_binarizer.fit_transform(y_test)

model3.compile(loss=categorical_crossentropy,
              optimizer=Adadelta(),
              metrics  =['accuracy'])
history = model3.fit(X_normalized, 
                    y_one_hot, epochs=3,
                    validation_split=0.2)

print('----------------------------------------------------------------------')
score = model3.evaluate(X_test2, y_test_onehot, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 80 samples, validate on 20 samples
Epoch 1/3
80/80 [==============================] - 1s 9ms/step - loss: 1.3032 - acc: 0.3125 - val_loss: 0.9203 - val_acc: 0.6000
Epoch 2/3
80/80 [==============================] - 0s 705us/step - loss: 0.9537 - acc: 0.5750 - val_loss: 0.8471 - val_acc: 0.6500
Epoch 3/3
80/80 [==============================] - 0s 623us/step - loss: 0.8568 - acc: 0.6375 - val_loss: 0.6531 - val_acc: 0.8500
----------------------------------------------------------------------
Test loss: 0.842922031879425
Test accuracy: 0.75


# Dropout

- Build from the previous network.
- Add a dropout layer after the pooling layer. 
    - Set the dropout rate to 50%.
- Make sure to note from the documentation above that the rate specified for dropout in Keras is the opposite of TensorFlow.
    - TensorFlow uses the probability to keep nodes. Keras uses the probability to drop them.
    

In [47]:
from keras.layers.core import Dropout

model4 = Sequential()
model4.add(Conv2D(32, (3,3), input_shape=(32, 32, 3), activation='relu'))
model4.add(MaxPooling2D((2, 2)))

# Before or after Flatten layer?
model4.add(Flatten())
model4.add(Dropout(0.5))
model4.add(Dense(128, activation='relu'))
model4.add(Dense(5, activation='softmax'))
label_binarizer = LabelBinarizer()
y_one_hot = label_binarizer.fit_transform(y_train)
y_test_onehot = label_binarizer.fit_transform(y_test)

model4.compile(loss=categorical_crossentropy,
              optimizer=Adadelta(),
              metrics  =['accuracy'])
history = model4.fit(X_normalized, 
                    y_one_hot, epochs=3,
                    validation_split=0.2)

print('----------------------------------------------------------------------')
score = model4.evaluate(X_test2, y_test_onehot, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 80 samples, validate on 20 samples
Epoch 1/3
80/80 [==============================] - 1s 13ms/step - loss: 1.4460 - acc: 0.3125 - val_loss: 0.9682 - val_acc: 0.7000
Epoch 2/3
80/80 [==============================] - 0s 795us/step - loss: 1.0108 - acc: 0.5625 - val_loss: 0.7567 - val_acc: 0.7000
Epoch 3/3
80/80 [==============================] - 0s 715us/step - loss: 0.8204 - acc: 0.7000 - val_loss: 0.6232 - val_acc: 0.7500
----------------------------------------------------------------------
Test loss: 0.7863947153091431
Test accuracy: 0.699999988079071


In [38]:
15*15*16

3600